In [2]:
import tensorflow as tf

In [3]:
#download data
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/",one_hot=True)

Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz


In [4]:
# shapes
mnist.train.labels.shape,mnist.train.images.shape,\
mnist.test.labels.shape,mnist.test.images.shape

((55000, 10), (55000, 784), (10000, 10), (10000, 784))

In [5]:
n_output_layer=10
def convolutional_neural_network(data):
    weights={"w_conv1":tf.Variable(tf.random_normal([5,5,1,32])),
            "w_conv2":tf.Variable(tf.random_normal([5,5,32,64])),
            "w_fc":tf.Variable(tf.random_normal([7*7*64,1024])),
            "out":tf.Variable(tf.random_normal([1024,n_output_layer]))}
    biases={"b_conv1":tf.Variable(tf.random_normal([32])),
           "b_conv2":tf.Variable(tf.random_normal([64])),
           "b_fc":tf.Variable(tf.random_normal([1024])),
           "out":tf.Variable(tf.random_normal([n_output_layer]))}
    data=tf.reshape(data,[-1,28,28,1])
    
    conv1=tf.nn.relu(tf.add(tf.nn.conv2d(data,weights["w_conv1"],
        strides=[1,1,1,1],padding="SAME"),biases["b_conv1"]))
    conv1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],
                        padding="SAME")
    conv2=tf.nn.relu(
            tf.add(tf.nn.conv2d(conv1,weights["w_conv2"],
        strides=[1,1,1,1],padding="SAME"),biases["b_conv2"]))
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    fc = tf.reshape(conv2, [-1, 7 * 7 * 64])
    fc = tf.nn.relu(tf.add(tf.matmul(fc, weights['w_fc']), biases['b_fc']))
    output=tf.add(tf.matmul(fc,weights["out"]),biases["out"])
    return output

In [6]:
batch_size=100
X=tf.placeholder("float",[None,28*28])
Y=tf.placeholder("float")

In [7]:
def train_neural_network(X,Y):
    predict=convolutional_neural_network(X)
    cost_func=tf.reduce_mean(tf.nn
                             .softmax_cross_entropy_with_logits(predict,Y))
    optimizer=tf.train.AdamOptimizer().minimize(cost_func)   
    
    epochs=1
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())
        epoch_loss=0
        for epoch in range(epochs):
            for i in range(int(mnist.train.num_examples/batch_size)):
                x,y=mnist.train.next_batch(batch_size)
                _,c=session.run([optimizer,cost_func],feed_dict={X:x,Y:y})
                epoch_loss+=c
                if not i%100:
                    print(epoch,":",epoch_loss)
                    correct_vector=tf.equal(tf.argmax(predict,1),tf.argmax(Y,1))
                    accuray=tf.reduce_mean(tf.cast(correct_vector,"float"))
                    print("accuray:",accuray.eval({X:mnist.test.images,Y:mnist.test.labels}))
train_neural_network(X,Y) 

0 : 60417.1796875
accuray: 0.1508
0 : 775272.762939
accuray: 0.832
0 : 967669.355957
accuray: 0.8856
0 : 1102471.52781
accuray: 0.9185
0 : 1220499.4435
accuray: 0.9327
0 : 1308356.97851
accuray: 0.9337
